## Step 1 import necessary module and load data

In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display


# Remember change to your own path
path = "Documents/p5/data/"
train = scipy.io.loadmat(path + 'train_32x32.mat')
test = scipy.io.loadmat(path + 'test_32x32.mat')

x_train = train['X']
y_train = train['y']
x_test = test['X']
y_test = test['y']

## Step 2 pickup several training data, plot them out to check image quality

In [ ]:
# Some image is very blur, like this one 
im = x_train.T[890].T.reshape(32,32,3)

plt.imshow(im)
plt.show()

print y_train[1667]
print x_train.shape

In [ ]:
# Reshape the training data and testing data
def reformat(data):
    lenth = data['X'].shape[3]
    X_train = []
    y_train = np.zeros((lenth,10))
    
    for i in range(lenth):
        X_train.append(data['X'][:,:,:,i])
        y_train[i][data['y'][i]%10] = 1
    
    return np.array(X_train).astype('float32') ,np.array(y_train).astype('float32')

X_train,y_train = reformat(train)
X_test,y_test = reformat(test)

In [ ]:
## check image is still good after reshape
img = X_train[42,:,:,:]
plt.imshow(img)
plt.show()

y_train[42]

In [ ]:
plt.figure(figsize = (30,20))

for i in range(50):
    plt.subplot(5,10,1+i)
    plt.axis('off')
    plt.imshow(X_train[i*5,:,:,:],cmap='Greys')

## Step 3 build up LeNet graph

In [ ]:
"""
Various Hyperparameters required for training the CNN.
"""
image_size = 32
width = 32
height = 32
channels = 3

n_labels = 10
patch = 5
depth = 16
hidden = 128

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
def model(data):
    #   Convolution 1 and RELU
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    #   Max Pool
    hidden2 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #   Convolution 2 and RELU
    conv2 = tf.nn.conv2d(hidden2, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden3 = tf.nn.relu(conv2 + layer2_biases)
    #   Max Pool
    hidden4 = tf.nn.max_pool(hidden3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = hidden4.get_shape().as_list()

    reshape = tf.reshape(hidden4, [-1, shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #   Dropout
    dropout_layer = tf.nn.dropout(hidden5, 0.93)

    return tf.matmul(dropout_layer, layer4_weights) + layer4_biases

logits = model(tf_train_dataset)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

train_prediction = tf.nn.softmax(logits)
test_prediction = tf.nn.softmax(model(tf_test_dataset))

## Step 4 Train this network and save the final model to hardisk

In [ ]:
t0 = time.time()
#   Number of iterations
saver = tf.train.Saver()

with tf.Session() as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average = 0
    for step in range(num_steps):
        #   Constucting the batch from the data set
        offset = (step * batch) % (y_train.shape[0] - batch)
        batch_data = X_train[offset:(offset + batch), :, :, :]
        batch_labels = y_train[offset:(offset + batch), :]
        #   Dictionary to be fed to TensorFlow Session
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout: drop_out}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        #   Calculating the Accuracy of the predictions
        accu = accuracy(predictions, batch_labels)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accu)
        average += accu
    print "Average Accuracy : ", (average / num_steps)
    print "END OF TRAINING"
    average = 0
    for step in range(num_steps):
        #   Constucting the batch from the data set
        offset = (step * batch) % (y_test.shape[0] - batch)
        batch_data = X_test[offset:(offset + batch), :, :, :]
        batch_labels = y_test[offset:(offset + batch), :]
        #   Dictionary to be fed to TensorFlow Session
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout: drop_out}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        #   Calculating the Accuracy of the predictions
        accu = accuracy(predictions, batch_labels)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accu)
        average += accu
    print "Average Accuracy : ", (average / num_steps)
    print "END OF TESTING"
    feed_dict = {tf_train_dataset : [img]}
    classification = session.run(train_prediction, feed_dict)
    print classification
    save_path = saver.save(session, "model.ckpt")

print "Training time: " + str(time.time()-t0)